In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from Keys import CLIENT_ID, CLIENT_SECRET
import time
import pandas as pd
import os

In [2]:
# specifies the path to the chromedriver.exe
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {
"download.default_directory": r"C:\Users\gacuf\Documents\MSAI\CS5100 Foundations of Artificial Intelligence\Resume Parser\LinkedIn\data\English Teachers Profiles",
"download.prompt_for_download": False,
"download.directory_upgrade": True
}

options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome('chromedriver',chrome_options=options)

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com')
time.sleep(1)

C:\Users\gacuf\AppData\Local\Temp\ipykernel_15012\2029203760.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=options)


In [3]:
# locate email form by_class_name
username = driver.find_element_by_id('session_key')

# send_keys() to simulate key strokes
username.send_keys(CLIENT_ID)

In [4]:
# locate password form by_class_name
password = driver.find_element_by_id('session_password')

# send_keys() to simulate key strokes
password.send_keys(CLIENT_SECRET)

In [5]:
# locate submit button by_class_name
log_in_button = driver.find_element(By.XPATH, "//button[@data-id='sign-in-form__submit-btn']")

# .click() to mimic button click
log_in_button.click()

In [ ]:
# Get Job listing for machine learning in boston entry level
url = "https://www.linkedin.com/jobs/search/?f_CR=103644278&f_E=2&geoId=102380872&keywords=machine%20learning&location=Boston%2C%20Massachusetts%2C%20United%20States&refresh=true&sortBy=R"
driver.get(url)
time.sleep(3)

In [ ]:
job_list_xpath_len = 0
job_list_xpath = driver.find_elements_by_xpath("//div[contains(@class, 'clickable')]")

while len(job_list_xpath) > job_list_xpath_len: 
    time.sleep(1)
    job_list_xpath_len = len(job_list_xpath)
    
    driver.execute_script("arguments[0].scrollIntoView();", job_list_xpath[-1])
    time.sleep(1)
    job_list_xpath = driver.find_elements_by_xpath("//div[contains(@class, 'clickable')]")

In [ ]:
job_description = []
company_urls = []
time.sleep(3)
company_name = []
job_title = []

for job in job_list_xpath:
    job.click()
    time.sleep(1)
    
    text = driver.find_element_by_class_name("jobs-box__html-content.jobs-description-content__text.t-14.t-normal.jobs-description-content__text--stretch").text
    job_description.append(text)
    
    title = driver.find_element_by_class_name("t-24.t-bold.jobs-unified-top-card__job-title").text
    job_title.append(title)
    
    name = driver.find_element_by_class_name("jobs-unified-top-card__company-name").text
    company_name.append(name)
    
    company_url = driver.find_element_by_class_name("ember-view.link-without-visited-state.inline-block.t-black")
    company_urls.append(company_url.get_attribute("href").replace("/life/","/people/?facetGeoRegion=103644278&keywords=machine%20learning"))

In [ ]:
jobs_df = pd.DataFrame()
jobs_df["Company"] = pd.Series(company_name)
jobs_df["URL"] = pd.Series(company_urls)
jobs_df["Title"] = pd.Series(job_title)
jobs_df["Description"] = pd.Series(job_description)

jobs_df.to_csv("data/MLJobs.csv")

In [ ]:
profiles = []
company_name = []
checked = []

for i, job in jobs_df.iterrows():
    if job[0] in checked:
        continue
        
    try:
        driver.get(job[1])
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        people_list = driver.find_elements_by_class_name("display-flex.list-style-none.flex-wrap")
        people_urls = people_list[0].find_elements_by_class_name("app-aware-link")
        for profile in people_urls:
            profiles.append(profile.get_attribute("href"))
            company_name.append(job[0])
        
        checked.append(job[0])
       
    except:
        print("No employees for: ", job[0])
    

In [ ]:
profiles_df = pd.DataFrame()
profiles_df["Company"] = pd.Series(company_name)
profiles_df["Profile"] = pd.Series(profiles)
profiles_df = profiles_df.drop_duplicates().reset_index(drop=True)
profiles_df.to_csv("data/MLProfiles_URL.csv")

In [ ]:
profile_df = pd.read_csv("data/MLProfiles_URL.csv")
errors = 0

for profile in profiles_df["Profile"]:
    
    driver.get(profile)
    time.sleep(1)

    # Name
    name = driver.find_element_by_class_name("text-heading-xlarge.inline.t-24.v-align-middle.break-words").text
    print("Downloading: ", name)
    
    # More
    more = driver.find_elements_by_class_name("artdeco-dropdown__trigger.artdeco-dropdown__trigger--placement-bottom.ember-view.pvs-profile-actions__action.artdeco-button.artdeco-button--secondary.artdeco-button--muted.artdeco-button--2")[1]
    more.click()
    time.sleep(2)

    # PDF
    content = "artdeco-dropdown__content-inner"
    overflow = driver.find_elements_by_class_name(content)
    overflow[1].find_elements_by_tag_name("li")[1].click()
    
    time.sleep(3)
    
    try:
        # Rename
        if errors == 0:
            old_name = os.path.join("data","Profile.pdf")
            new_name = os.path.join("data",name + ".pdf")
            os.rename(old_name,new_name)
        else:
            old_name = os.path.join("data","Profile (" + str(errors) + ").pdf")
            new_name = os.path.join("data",name + ".pdf")
            os.rename(old_name,new_name)
            
    
    except Exception as e: 
        print(e)
        errors +=1

In [ ]:
# Get Job listing for painters in boston entry level
url = "https://www.linkedin.com/jobs/search/?geoId=102380872&keywords=painter%20painting&location=Boston%2C%20Massachusetts%2C%20United%20States&refresh=true"
driver.get(url)
time.sleep(3)

In [ ]:
job_list_xpath_len = 0
job_list_xpath = driver.find_elements_by_xpath("//div[contains(@class, 'clickable')]")

while len(job_list_xpath) > job_list_xpath_len: 
    time.sleep(1)
    job_list_xpath_len = len(job_list_xpath)
    
    driver.execute_script("arguments[0].scrollIntoView();", job_list_xpath[-1])
    time.sleep(1)
    job_list_xpath = driver.find_elements_by_xpath("//div[contains(@class, 'clickable')]")

In [ ]:
job_description = []
company_urls = []
time.sleep(3)
company_name = []
job_title = []

for job in job_list_xpath:
    job.click()
    time.sleep(1)
    
    text = driver.find_element_by_class_name("jobs-box__html-content.jobs-description-content__text.t-14.t-normal.jobs-description-content__text--stretch").text
    job_description.append(text)
    
    title = driver.find_element_by_class_name("t-24.t-bold.jobs-unified-top-card__job-title").text
    job_title.append(title)
    
    name = driver.find_element_by_class_name("jobs-unified-top-card__company-name").text
    company_name.append(name)
    
    company_url = driver.find_element_by_class_name("ember-view.link-without-visited-state.inline-block.t-black")
    company_urls.append(company_url.get_attribute("href").replace("/life/","/people/?facetGeoRegion=103644278&keywords=painter%20painting"))

In [ ]:
jobs_df = pd.DataFrame()
jobs_df["Company"] = pd.Series(company_name)
jobs_df["URL"] = pd.Series(company_urls)
jobs_df["Title"] = pd.Series(job_title)
jobs_df["Description"] = pd.Series(job_description)

jobs_df.to_csv("data/PaintingJobs.csv")

In [ ]:
profiles = []
company_name = []
checked = []

for i, job in jobs_df.iterrows():
    if job[0] in checked:
        continue
        
    try:
        driver.get(job[1])
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        people_list = driver.find_elements_by_class_name("display-flex.list-style-none.flex-wrap")
        people_urls = people_list[0].find_elements_by_class_name("app-aware-link")
        for profile in people_urls:
            profiles.append(profile.get_attribute("href"))
            company_name.append(job[0])
        
        checked.append(job[0])
       
    except:
        print("No employees for: ", job[0])
    

In [ ]:
profiles_df = pd.DataFrame()
profiles_df["Company"] = pd.Series(company_name)
profiles_df["Profile"] = pd.Series(profiles)
profiles_df = profiles_df.drop_duplicates().reset_index(drop=True)
profiles_df.to_csv("data/PaintingProfiles_URL.csv")

In [7]:
profiles_df = pd.read_csv("data/PaintingProfiles_URL.csv")
errors = 0

for profile in profiles_df["Profile"][:12]:
    
    driver.get(profile)
    time.sleep(1)

    # Name
    name = driver.find_element_by_class_name("text-heading-xlarge.inline.t-24.v-align-middle.break-words").text
    print("Downloading: ", name)
    
    # More
    more = driver.find_elements_by_class_name("artdeco-dropdown__trigger.artdeco-dropdown__trigger--placement-bottom.ember-view.pvs-profile-actions__action.artdeco-button.artdeco-button--secondary.artdeco-button--muted.artdeco-button--2")[1]
    more.click()
    time.sleep(2)

    # PDF
    content = "artdeco-dropdown__content-inner"
    overflow = driver.find_elements_by_class_name(content)
    overflow[1].find_elements_by_tag_name("li")[1].click()
    
    time.sleep(3)
    
    try:
        # Rename
        if errors == 0:
            old_name = os.path.join("data","Profile.pdf")
            new_name = os.path.join("data",name + ".pdf")
            os.rename(old_name,new_name)
        else:
            old_name = os.path.join("data","Profile (" + str(errors) + ").pdf")
            new_name = os.path.join("data",name + ".pdf")
            os.rename(old_name,new_name)
            
    
    except Exception as e: 
        print(e)
        errors +=1

Downloading:  Donna McNally MSW, LICSW, LADC1
[WinError 2] The system cannot find the file specified: 'data\\Profile.pdf' -> 'data\\Donna McNally MSW, LICSW, LADC1.pdf'
Downloading:  Kristian Perry
[WinError 2] The system cannot find the file specified: 'data\\Profile (1).pdf' -> 'data\\Kristian Perry.pdf'
Downloading:  Thomas Lindberg
[WinError 2] The system cannot find the file specified: 'data\\Profile (2).pdf' -> 'data\\Thomas Lindberg.pdf'
Downloading:  Carlos Ruiz
[WinError 2] The system cannot find the file specified: 'data\\Profile (3).pdf' -> 'data\\Carlos Ruiz.pdf'
Downloading:  Teresa Scott
[WinError 2] The system cannot find the file specified: 'data\\Profile (4).pdf' -> 'data\\Teresa Scott.pdf'
Downloading:  Steven Hudspeth Owner of TMO Turbo
[WinError 2] The system cannot find the file specified: 'data\\Profile (5).pdf' -> 'data\\Steven Hudspeth Owner of TMO Turbo.pdf'
Downloading:  Sharon Strange
[WinError 2] The system cannot find the file specified: 'data\\Profile (6).

KeyboardInterrupt: 

In [ ]:
# Get Job listing for english teachers in boston entry level
url = "https://www.linkedin.com/jobs/search/?currentJobId=3507639114&geoId=102380872&keywords=english%20teacher&location=Boston%2C%20Massachusetts%2C%20United%20States&refresh=true"
driver.get(url)
time.sleep(3)

In [ ]:
# Get list element
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#jobs_list_div = driver.find_element_by_class_name("jobs-search-results-list")
#driver.execute_script()
jobs_class_name = "job-card-container.relative.job-card-list.job-card-container--clickable.job-card-list--underline-title-on-hover.jobs-search-two-pane__job-card-container--viewport-tracking-10"
job_list = driver.find_element_by_class_name(jobs_class_name)
job_list

In [ ]:
# Contactos
contactos = driver.find_elements_by_class_name('t-16.t-black.t-normal')[1].text
contactos

In [ ]:
# Ubicación
ubicacion = driver.find_elements_by_class_name('t-16.t-black.t-normal')[0].text
ubicacion

In [ ]:
# Name
nombre = driver.find_element_by_class_name('inline.t-24.t-black.t-normal.break-words').text
nombre

In [ ]:
# Foto Cara
cara = driver.find_element_by_class_name('pv-top-card__photo.presence-entity__image').get_attribute("src")
cara
# sin foto: data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7

In [ ]:
# Foto Fondo
try:
    fondo = driver.find_element_by_class_name('profile-background-image__image').get_attribute("src")
    print(fondo)
except:
    print('Sin fondo')
# sin foto:

In [ ]:
# Cargo
cargo = driver.find_element_by_class_name('mt1.t-18.t-black.t-normal.break-words').text
cargo

In [ ]:
# Último Empleo
ultimo_empleo = driver.find_elements_by_class_name('text-align-left.ml2.t-14.t-black.t-bold')[0].text
ultimo_empleo

In [ ]:
# Último Estudio
ultimo_estudio = driver.find_elements_by_class_name('text-align-left.ml2.t-14.t-black.t-bold')[1].text
ultimo_estudio

In [ ]:
# Extracto
extracto_texto = ''
try:
    driver.find_element_by_id('line-clamp-show-more-button').click()
    extracto = driver.find_elements_by_class_name('lt-line-clamp__raw-line')
    for i in range(len(extracto)):
        extracto_texto = extracto_texto + extracto[i].text
    print(extracto_texto)
except:
    print('Sin ver mas')
    extracto = driver.find_elements_by_class_name('lt-line-clamp__line')
    if len(extracto)==0:
        print('Sin extracto')

In [ ]:
# More
more = driver.find_elements_by_class_name('ml2')[2]
more.click()

In [ ]:
# Save to PDF
save = 'pv-s-profile-actions.pv-s-profile-actions--save-to-pdf'
save_pdf = driver.find_element_by_class_name(save).click()

In [ ]:
time.sleep(2)
driver.quit()